<h1 align=center><font size = 5>Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá</font></h1>


## Seccion 1


#### rastreo web de wikipedia


https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#### primera parte

In [1]:
!pip install geopy

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.7 MB/s eta 0:00:00


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing th

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
table = soup.find('table')

In [3]:
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): 
        cols = row.find_all('p') 
        file.append(cols[i].getText())

df= pd.DataFrame(file)

In [4]:
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)[0]
df["Neighborhood"]=df["Neigh"].str.replace(" / ",",")
postal_list=df[(df["Borough"]!="Not assigned\n")]
postal_list=postal_list.drop([0,"Neigh"],axis=1)
postal_list.head(10)

,Postal Code,Borough,Neighborhood
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
8,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
9,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"
10,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [5]:
postal_list.shape

(103, 3)

### segunda parte

In [6]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data

Geospatial_Coordinates=pd.read_csv("Geospatial_Coordinates.csv")


In [7]:
coord_list=pd.merge(postal_list,Geospatial_Coordinates,on="Postal Code")
coord_list.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### tercera parte

In [8]:
address = 'TORONTO,CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas geograficas de Toronto son {}, {}.'.format(latitude, longitude))

Las coordenadas geograficas de Toronto son 43.6534817, -79.3839347.


In [9]:
toronto_data = coord_list[coord_list['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.groupby("Borough").size().sort_values(ascending=False)


Borough
Downtown Toronto                                                17
Central Toronto                                                  9
West Toronto                                                     6
East Toronto                                                     4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
dtype: int64

In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
CLIENT_ID = '2OF3KKCYFHP50KM33SYTWRWORAU5A2IWLQGYH2K3NRPGTTMU' # su ID de Foursquare
CLIENT_SECRET = 'TSQBJ2TIQLG5KFUSS234GQO5KGMFVWL5FVKWVV5KYVQJUPXB' # su Secreto de Cliente de Foursquare
#VERSION = '20180604'
VERSION = '20230417'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2OF3KKCYFHP50KM33SYTWRWORAU5A2IWLQGYH2K3NRPGTTMU
CLIENT_SECRET:TSQBJ2TIQLG5KFUSS234GQO5KGMFVWL5FVKWVV5KYVQJUPXB


In [12]:
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [13]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # latitud del barrio 
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # longitud del barrio

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # nombre del barrio

print('Latitude and longitude values of {} are {},{}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.6763574,-79.2930312.


In [41]:
LIMIT= 50
radius= 500
url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&limit={}".format(neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v3/places/search?ll=43.685347%2C-79.3381065&radius=500&limit=50'

In [42]:

#url = "https://api.foursquare.com/v3/places/search?ll=43.6763574%2C%20-79.2930312&radius=500&limit=50"
#url = "https://api.foursquare.com/v3/places/search?ll={}%2C%20{}&radius={}&limit={}".format(neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

headers = {
    "accept": "application/json",
    "Authorization": "fsq3+btNZJiWH0LtffL3n4zu2NiE6iThx9A752cLEJs5q1U="
}
#response = requests.get(url, headers=headers)
#results = requests.get(url, headers=headers)
results = requests.get(url, headers=headers).json()
results
#print(results.text)

{'results': [{'fsq_id': '4ce01b73ffcf370418c21d82',
   'categories': [{'id': 16037,
     'name': 'Playground',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 112,
   'geocodes': {'drop_off': {'latitude': 43.686032, 'longitude': -79.337428},
    'main': {'latitude': 43.686093, 'longitude': -79.337289},
    'roof': {'latitude': 43.686093, 'longitude': -79.337289}},
   'link': '/v3/places/4ce01b73ffcf370418c21d82',
   'location': {'address': '3 Leroy Ave',
    'country': 'CA',
    'cross_street': '',
    'formatted_address': '3 Leroy Ave, Toronto ON M4J 4G6',
    'locality': 'Toronto',
    'postcode': 'M4J 4G6',
    'region': 'ON'},
   'name': "Gabe's Playground",
   'related_places': {},
   'timezone': 'America/Toronto'},
  {'fsq_id': '5a8880cd22d4902a7355018a',
   'categories': [{'id': 17135,
     'name': 'Toy / Game Store',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categorie

In [44]:
venues = results['results']
nearby_venues = json_normalize(venues) # objeto JSON

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [45]:
nearby_venues.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.drop_off.latitude,geocodes.drop_off.longitude,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region
0,4ce01b73ffcf370418c21d82,"[{'id': 16037, 'name': 'Playground', 'icon': {...",[],112,/v3/places/4ce01b73ffcf370418c21d82,Gabe's Playground,America/Toronto,43.686032,-79.337428,43.686093,-79.337289,43.686093,-79.337289,3 Leroy Ave,CA,,"3 Leroy Ave, Toronto ON M4J 4G6",Toronto,M4J 4G6,ON
1,5a8880cd22d4902a7355018a,"[{'id': 17135, 'name': 'Toy / Game Store', 'ic...",[],137,/v3/places/5a8880cd22d4902a7355018a,Treehouse Collectibles,America/Toronto,43.686036,-79.339589,43.685971,-79.339749,43.685971,-79.339749,166 Donlands Ave,CA,,"166 Donlands Ave, Toronto ON M4J 3P8",Toronto,M4J 3P8,ON
2,50a404cce4b04c6e7950e679,"[{'id': 11062, 'name': 'Barbershop', 'icon': {...",[],141,/v3/places/50a404cce4b04c6e7950e679,Finesse Cut,America/Toronto,43.686036,-79.339589,43.685971,-79.339749,43.685971,-79.339749,164 Donlands Ave,CA,Donlands And Sammon Ave.,"164 Donlands Ave (Donlands And Sammon Ave.), T...",Toronto,M4J 3P8,ON
3,0e9a410c1ef544a538742e54,"[{'id': 19055, 'name': 'Travel Agency', 'icon'...",[],96,/v3/places/0e9a410c1ef544a538742e54,Royal Vacation Travel Tours,NaN,NaN,NaN,43.686176,-79.338472,43.686176,-79.338472,238 Sammon Ave,CA,NaN,"238 Sammon Ave, East York ON M4J 1Z5",East York,M4J 1Z5,ON
4,f3dc389bb2b74c3e6e87c8fc,"[{'id': 18000, 'name': 'Sports and Recreation'...",[],113,/v3/places/f3dc389bb2b74c3e6e87c8fc,Dc Sports,NaN,NaN,NaN,43.684718,-79.339224,43.684718,-79.339224,809 Loomis Crt,CA,NaN,"809 Loomis Crt, Toronto ON M4J 3P6",Toronto,M4J 3P6,ON


In [67]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['categories.name']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.drop_off.latitude,geocodes.drop_off.longitude,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region
0,4ce01b73ffcf370418c21d82,Playground,[],112,/v3/places/4ce01b73ffcf370418c21d82,Gabe's Playground,America/Toronto,43.686032,-79.337428,43.686093,-79.337289,43.686093,-79.337289,3 Leroy Ave,CA,,"3 Leroy Ave, Toronto ON M4J 4G6",Toronto,M4J 4G6,ON
1,5a8880cd22d4902a7355018a,Toy / Game Store,[],137,/v3/places/5a8880cd22d4902a7355018a,Treehouse Collectibles,America/Toronto,43.686036,-79.339589,43.685971,-79.339749,43.685971,-79.339749,166 Donlands Ave,CA,,"166 Donlands Ave, Toronto ON M4J 3P8",Toronto,M4J 3P8,ON
2,50a404cce4b04c6e7950e679,Barbershop,[],141,/v3/places/50a404cce4b04c6e7950e679,Finesse Cut,America/Toronto,43.686036,-79.339589,43.685971,-79.339749,43.685971,-79.339749,164 Donlands Ave,CA,Donlands And Sammon Ave.,"164 Donlands Ave (Donlands And Sammon Ave.), T...",Toronto,M4J 3P8,ON
3,0e9a410c1ef544a538742e54,Travel Agency,[],96,/v3/places/0e9a410c1ef544a538742e54,Royal Vacation Travel Tours,NaN,NaN,NaN,43.686176,-79.338472,43.686176,-79.338472,238 Sammon Ave,CA,NaN,"238 Sammon Ave, East York ON M4J 1Z5",East York,M4J 1Z5,ON
4,f3dc389bb2b74c3e6e87c8fc,Sports and Recreation,[],113,/v3/places/f3dc389bb2b74c3e6e87c8fc,Dc Sports,NaN,NaN,NaN,43.684718,-79.339224,43.684718,-79.339224,809 Loomis Crt,CA,NaN,"809 Loomis Crt, Toronto ON M4J 3P6",Toronto,M4J 3P6,ON


In [27]:
filtered_columns = ['name', 'categories', 'geocodes.main.latitude', 'geocodes.main.longitude']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,name,categories,geocodes.main.latitude,geocodes.main.longitude
0,Glen Manor Ravine,Hiking Trail,43.676676,-79.294126
1,Glen Stewart Park,Park,43.674403,-79.294516
2,Santon Audio,Audiovisual Service,43.677003,-79.294270
3,Loach Engineering Inc,Sports and Recreation,43.675253,-79.293621
4,Verite Films Inc,Film Studio,43.675897,-79.295593


In [28]:
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,latitude,longitude
0,Glen Manor Ravine,Hiking Trail,43.676676,-79.294126
1,Glen Stewart Park,Park,43.674403,-79.294516
2,Santon Audio,Audiovisual Service,43.677003,-79.294270
3,Loach Engineering Inc,Sports and Recreation,43.675253,-79.293621
4,Verite Films Inc,Film Studio,43.675897,-79.295593


In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&limit={}".format(lat, lng, radius, LIMIT)
        headers = {
        "accept": "application/json",
        "Authorization": "fsq3+btNZJiWH0LtffL3n4zu2NiE6iThx9A752cLEJs5q1U="
        }
            
        # solicitud GET
        results = requests.get(url, headers=headers).json()
        venues = results['results']

        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['geocodes']['main']['latitude'], 
            v['geocodes']['main']['longitude'],  
            (v['categories'][0]['name'] if len(v['categories']) > 0 else None)) for v in venues])
       


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
nearby_venues

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.drop_off.latitude,geocodes.drop_off.longitude,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region
0,4ce01b73ffcf370418c21d82,Playground,[],112,/v3/places/4ce01b73ffcf370418c21d82,Gabe's Playground,America/Toronto,43.686032,-79.337428,43.686093,-79.337289,43.686093,-79.337289,3 Leroy Ave,CA,,"3 Leroy Ave, Toronto ON M4J 4G6",Toronto,M4J 4G6,ON
1,5a8880cd22d4902a7355018a,Toy / Game Store,[],137,/v3/places/5a8880cd22d4902a7355018a,Treehouse Collectibles,America/Toronto,43.686036,-79.339589,43.685971,-79.339749,43.685971,-79.339749,166 Donlands Ave,CA,,"166 Donlands Ave, Toronto ON M4J 3P8",Toronto,M4J 3P8,ON
2,50a404cce4b04c6e7950e679,Barbershop,[],141,/v3/places/50a404cce4b04c6e7950e679,Finesse Cut,America/Toronto,43.686036,-79.339589,43.685971,-79.339749,43.685971,-79.339749,164 Donlands Ave,CA,Donlands And Sammon Ave.,"164 Donlands Ave (Donlands And Sammon Ave.), T...",Toronto,M4J 3P8,ON
3,0e9a410c1ef544a538742e54,Travel Agency,[],96,/v3/places/0e9a410c1ef544a538742e54,Royal Vacation Travel Tours,NaN,NaN,NaN,43.686176,-79.338472,43.686176,-79.338472,238 Sammon Ave,CA,NaN,"238 Sammon Ave, East York ON M4J 1Z5",East York,M4J 1Z5,ON
4,f3dc389bb2b74c3e6e87c8fc,Sports and Recreation,[],113,/v3/places/f3dc389bb2b74c3e6e87c8fc,Dc Sports,NaN,NaN,NaN,43.684718,-79.339224,43.684718,-79.339224,809 Loomis Crt,CA,NaN,"809 Loomis Crt, Toronto ON M4J 3P6",Toronto,M4J 3P6,ON
5,e0c010db06c65faf3636a849,General Contractor,[],129,/v3/places/e0c010db06c65faf3636a849,Golden Fingers Janitorial Svc,NaN,43.686363,-79.337013,43.686171,-79.336974,43.686171,-79.336974,271 Sammon Ave,CA,NaN,"271 Sammon Ave, East York ON M4J 1Z8",East York,M4J 1Z8,ON
6,39ddd7423b564da76935dbef,Jewelry Store,[],135,/v3/places/39ddd7423b564da76935dbef,Victoria's Finest,NaN,NaN,NaN,43.684735,-79.336653,43.684735,-79.336653,15 Wiley Ave,CA,NaN,"15 Wiley Ave, East York ON M4J 3W3",East York,M4J 3W3,ON
7,4a0de15c8ed447f70c5b9c5b,Vintage and Thrift Store,[],155,/v3/places/4a0de15c8ed447f70c5b9c5b,Treehouse Auctions,NaN,NaN,NaN,43.685965,-79.339842,43.685965,-79.339842,198A Sammon Ave,CA,NaN,"198A Sammon Ave, East York ON M4J 1Z2",East York,M4J 1Z2,ON
8,95dc65c68b4d4f4e956e009d,Landscaper and Gardener,[],155,/v3/places/95dc65c68b4d4f4e956e009d,Symmetry Landscape Design & Build,NaN,43.684344,-79.336693,43.684271,-79.336861,43.684271,-79.336861,8 Wiley Ave,CA,NaN,"8 Wiley Ave, East York ON M4J 3W4",East York,M4J 3W4,ON
9,85b3494125804cc6a8daca5f,Event Service,[],157,/v3/places/85b3494125804cc6a8daca5f,Your Wedding Dress Specialist,NaN,43.686357,-79.337041,43.686557,-79.337082,43.686557,-79.337082,280 Sammon Ave,CA,NaN,"280 Sammon Ave, Toronto ON M4J 1Z7",Toronto,M4J 1Z7,ON


In [77]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth  East
The Danforth West,Riverdale
India Bazaar,The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Summerhill West,Rathnelly,South Hill,Forest Hill SE,Deer Park
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Regent Park,Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond,Adelaide,King
Harbourfront East,Union Station,Toronto Islands
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North & West
The Annex,North Midtown,Yorkville
University of Toronto,Harbord
Kensington Market,Chinatown,Grange Park
CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport
Enclave of M5E
First Canadian Place,Underground city
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Brockton,Parkdale Village,Exhibition Place
High Park,The Junction South
Parkdale,Roncesvalles
Runn

In [79]:
print(toronto_venues.shape)
toronto_venues.head()

(1789, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676676,-79.294126,Hiking Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.674403,-79.294516,Park
2,The Beaches,43.676357,-79.293031,Santon Audio,43.677003,-79.294270,Audiovisual Service
3,The Beaches,43.676357,-79.293031,Loach Engineering Inc,43.675253,-79.293621,Sports and Recreation
4,The Beaches,43.676357,-79.293031,Verite Films Inc,43.675897,-79.295593,Film Studio


In [80]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton,Parkdale Village,Exhibition Place",50,50,50,50,50,48
"CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport",12,12,12,12,12,8
Central Bay Street,50,50,50,50,50,50
Christie,50,50,50,50,50,46
Church and Wellesley,50,50,50,50,50,50
"Commerce Court,Victoria Hotel",50,50,50,50,50,50
Davisville,50,50,50,50,50,49
Davisville North,50,50,50,50,50,45


In [82]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 321 uniques categories.


In [83]:
# codificación
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Art Gallery,...,Utility Company,Vegan and Vegetarian Restaurant,Video Games Store,Video Store,Vietnamese Restaurant,Vintage and Thrift Store,Website Designer,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
toronto_onehot.shape

(1789, 321)

In [85]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Art Gallery,...,Utility Company,Vegan and Vegetarian Restaurant,Video Games Store,Video Store,Vietnamese Restaurant,Vintage and Thrift Store,Website Designer,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.000000,0.00,0.00,0.02,0.02,0.00,0.000000,0.00,...,0.00,0.02,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00
1,"Brockton,Parkdale Village,Exhibition Place",0.00,0.000000,0.06,0.00,0.00,0.00,0.00,0.020000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00
2,"CN Tower,King and Spadina,Railway Lands,Harbou...",0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00
3,Central Bay Street,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00
4,Christie,0.00,0.000000,0.02,0.00,0.00,0.02,0.02,0.020000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.02,0.00,0.00
5,Church and Wellesley,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00
6,"Commerce Court,Victoria Hotel",0.00,0.000000,0.02,0.00,0.00,0.02,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00
7,Davisville,0.00,0.000000,0.00,0.00,0.00,0.02,0.04,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.020000,0.00000,0.00,0.00,0.00
8,Davisville North,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.02,0.00,0.00,0.00,0.00,0.000000,0.02000,0.00,0.00,0.00
9,"Dufferin,Dovercourt Village",0.00,0.020000,0.02,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00


In [86]:
toronto_grouped.shape

(39, 321)

In [87]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0          Pub  0.06
1  Coffee Shop  0.06
2    Drugstore  0.04
3          Bar  0.04
4     Beer Bar  0.04


----Brockton,Parkdale Village,Exhibition Place----
                venue  freq
0  Advertising Agency  0.06
1          Hair Salon  0.06
2   Financial Service  0.04
3                Café  0.04
4  Photography Studio  0.04


----CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport----
                         venue  freq
0     Home Improvement Service  0.08
1  Historic and Protected Site  0.08
2       Landmarks and Outdoors  0.08
3        Sports and Recreation  0.08
4              Harbor / Marina  0.08


----Central Bay Street----
                           venue  freq
0  Cafes, Coffee, and Tea Houses  0.12
1                           Deli  0.08
2                    Coffee Shop  0.08
3            Japanese Restaurant  0.06
4                           Café  0.06


----Christie----
                  

In [88]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [98]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Pub,Coffee Shop,Drugstore,Bar,Beer Bar,Night Club,Fish and Chips Shop,Farmers' Market,Sushi Restaurant,Clothing Store
1,"Brockton,Parkdale Village,Exhibition Place",Advertising Agency,Hair Salon,Financial Service,Café,Photography Studio,Coffee Shop,Computer Repair Service,Grocery Store / Supermarket,Park,Entertainment Service
2,"CN Tower,King and Spadina,Railway Lands,Harbou...",Home Improvement Service,Historic and Protected Site,Landmarks and Outdoors,Sports and Recreation,Harbor / Marina,Sculpture Garden,Car Dealership,Property Management Office,ATM,Museum
3,Central Bay Street,"Cafes, Coffee, and Tea Houses",Deli,Coffee Shop,Japanese Restaurant,Café,Fast Food Restaurant,Pizzeria,Diner,Sushi Restaurant,Mediterranean Restaurant
4,Christie,Grocery Store / Supermarket,Business and Professional Services,Restaurant,Hair Salon,General Contractor,Automotive Repair Shop,Café,Electrician,Community and Government,Computers and Electronics Retail


In [99]:
# establecer el número de agrupaciones
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any b

array([4, 1, 1, 2, 1, 0, 2, 0, 1, 1], dtype=int32)

In [100]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# juntar toronto_grouped con toronto_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # revisar las ultimas columnas

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Hair Salon,Interior Designer,Park,Dance Studio,Organization,Diner,General Contractor,Boutique,Pub,"Shipping, Freight, and Material Transportation..."
1,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,1,Business and Professional Services,Park,Convenience Store,Toy / Game Store,Automotive Repair Shop,Coffee Shop,Martial Arts Dojo,General Contractor,College and University,Sports and Recreation
2,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Restaurant,Ice Cream Parlor,Greek Restaurant,Diner,Café,Bookstore,Clothing Store,Retail,Bakery,Italian Restaurant
3,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572,1,Pizzeria,Café,Electronics Store,Bicycle Store,Advertising Agency,Movie Theater,Business and Professional Services,Park,Pet Grooming Service,Convenience Store
4,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Bar,Hair Salon,Italian Restaurant,Retail,Drugstore,Nail Salon,Coffee Shop,Fast Food Restaurant,Lounge,Bookstore


In [101]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Berczy Park,Pub,Coffee Shop,Drugstore,Bar,Beer Bar,Night Club,Fish and Chips Shop,Farmers' Market,Sushi Restaurant,Clothing Store
1,1,"Brockton,Parkdale Village,Exhibition Place",Advertising Agency,Hair Salon,Financial Service,Café,Photography Studio,Coffee Shop,Computer Repair Service,Grocery Store / Supermarket,Park,Entertainment Service
2,1,"CN Tower,King and Spadina,Railway Lands,Harbou...",Home Improvement Service,Historic and Protected Site,Landmarks and Outdoors,Sports and Recreation,Harbor / Marina,Sculpture Garden,Car Dealership,Property Management Office,ATM,Museum
3,2,Central Bay Street,"Cafes, Coffee, and Tea Houses",Deli,Coffee Shop,Japanese Restaurant,Café,Fast Food Restaurant,Pizzeria,Diner,Sushi Restaurant,Mediterranean Restaurant
4,1,Christie,Grocery Store / Supermarket,Business and Professional Services,Restaurant,Hair Salon,General Contractor,Automotive Repair Shop,Café,Electrician,Community and Government,Computers and Electronics Retail


In [102]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,0,Restaurant,Ice Cream Parlor,Greek Restaurant,Diner,Café,Bookstore,Clothing Store,Retail,Bakery,Italian Restaurant
4,East Toronto,0,Bar,Hair Salon,Italian Restaurant,Retail,Drugstore,Nail Salon,Coffee Shop,Fast Food Restaurant,Lounge,Bookstore
7,Central Toronto,0,Clothing Store,Nail Salon,Bicycle Store,Retail,Hair Salon,Health and Beauty Service,"Cafes, Coffee, and Tea Houses",Shoe Store,Cosmetics Store,Sushi Restaurant
8,Central Toronto,0,Flower Store,Nail Salon,Bank,Antique Store,Health and Beauty Service,Italian Restaurant,Japanese Restaurant,Toy / Game Store,Hobby Store,Bookstore
13,Downtown Toronto,0,Diner,Hair Salon,Clothing Store,Café,Italian Restaurant,Restaurant,Pizzeria,Liquor Store,Thai Restaurant,Tattoo Parlor
14,Downtown Toronto,0,Automotive Repair Shop,Park,Furniture and Home Store,Restaurant,Car Dealership,Italian Restaurant,Music Venue,Bakery,Café,Gourmet Store
16,Downtown Toronto,0,Café,Coffee Shop,Asian Restaurant,Burger Joint,Park,Fast Food Restaurant,Diner,Art Gallery,BBQ Joint,Bar
20,Downtown Toronto,0,Pizzeria,Music Venue,Discount Store,Deli,Bar,Plaza,Salad Restaurant,Park,Sports Bar,Café
26,Downtown Toronto,0,Café,Pizzeria,ATM,Bakery,Hot Dog Joint,Bookstore,College Administrative Building,Coffee Shop,Church,Sporting Goods Retail
27,Downtown Toronto,0,Café,Bakery,Chinese Restaurant,Vietnamese Restaurant,Barbershop,Bar,Comfort Food Restaurant,Tattoo Parlor,Organic Grocery,Gourmet Store


In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Hair Salon,Interior Designer,Park,Dance Studio,Organization,Diner,General Contractor,Boutique,Pub,"Shipping, Freight, and Material Transportation..."
1,East YorkEast Toronto,1,Business and Professional Services,Park,Convenience Store,Toy / Game Store,Automotive Repair Shop,Coffee Shop,Martial Arts Dojo,General Contractor,College and University,Sports and Recreation
3,East Toronto,1,Pizzeria,Café,Electronics Store,Bicycle Store,Advertising Agency,Movie Theater,Business and Professional Services,Park,Pet Grooming Service,Convenience Store
6,Central Toronto,1,Attorney / Law Office,Grocery Store / Supermarket,Business and Professional Services,Restaurant,Tailor,Playground,Pet Service,Hair Salon,Community and Government,Film Studio
9,Central Toronto,1,Business and Strategy Consulting Office,Tennis Court,Financial Planner,Business and Professional Services,Architecture Firm,Film Studio,Park,Media Agency,Bridge,Hair Salon
10,Central Toronto,1,Insurance Agency,Financial Planner,Telecommunication Service,Real Estate Agency,Sports and Recreation,Bank,Advertising Agency,Fabric / Textiles Store,Technology Business,Grocery Store / Supermarket
11,Downtown Toronto,1,Park,Carpet and Flooring Contractor,Interior Designer,Playground,Bike Trail,Property Management Office,Dry Cleaner,Real Estate Agency,Swimming Pool,Loans Agency
12,Downtown Toronto,1,Bakery,Pizzeria,Restaurant,Drugstore,Convenience Store,Pet Supplies Store,Night Club,Liquor Store,Loans Agency,Chinese Restaurant
24,Central Toronto,1,Business and Professional Services,General Contractor,Park,Restaurant,Gift Store,Electronics Store,Bar,Bank,Management Consultant,Caterer
25,Central Toronto,1,Café,Laundromat,Coffee Shop,Deli,Playground,Park,Yoga Studio,Middle Eastern Restaurant,Miscellaneous Store,Speakeasy


In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,2,Fast Food Restaurant,Coffee Shop,"Cafes, Coffee, and Tea Houses",Bubble Tea Shop,BBQ Joint,Restaurant,Department Store,Bar,Clothing Store,Garden
18,Downtown Toronto,2,"Cafes, Coffee, and Tea Houses",Deli,Coffee Shop,Japanese Restaurant,Café,Fast Food Restaurant,Pizzeria,Diner,Sushi Restaurant,Mediterranean Restaurant
19,Downtown Toronto,2,Deli,Restaurant,Café,"Cafes, Coffee, and Tea Houses",Hotel Bar,Breakfast Spot,Coffee Shop,American Restaurant,Bakery,Brazilian Restaurant
21,Downtown Toronto,2,Restaurant,"Cafes, Coffee, and Tea Houses",Bank,Deli,Café,Coffee Shop,Diner,Bakery,Drugstore,Pub
22,Downtown Toronto,2,"Cafes, Coffee, and Tea Houses",Deli,Bank,Fast Food Restaurant,Café,Burger Joint,Bistro,Bakery,Steakhouse,Drugstore
30,Downtown Toronto,2,"Cafes, Coffee, and Tea Houses",Bank,Restaurant,Deli,Café,Thai Restaurant,Bakery,Drugstore,Food Court,Hotel Bar


In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,3,Business and Strategy Consulting Office,Architecture Firm,Hobby Store,Dim Sum Restaurant,Management Consultant,Legal Service,Tennis Club,Spa,Bakery,Organization
23,Central Toronto,3,Construction,Business and Professional Services,Computer Repair Service,Business and Strategy Consulting Office,Hockey,Accounting and Bookkeeping Service,Interior Designer,Landscaper and Gardener,General Contractor,Mover


In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,4,Pub,Coffee Shop,Drugstore,Bar,Beer Bar,Night Club,Fish and Chips Shop,Farmers' Market,Sushi Restaurant,Clothing Store
29,Downtown TorontoStn A PO Boxes25 The Esplanade,4,Pub,Italian Restaurant,Drugstore,Night Club,Coffee Shop,French Restaurant,Fish and Chips Shop,Music Venue,Beer Bar,Bistro
